# Imports Librerias

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [4]:
import pandas as pd
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# Funciones Visualización de Datos:

In [5]:
def countClassesFunc(data, atribute, printClasses = False, plotBar = False):
  countClasses = {}
  for cls in data[atribute].unique():
    countClasses[cls] = sum(data[atribute].values==cls)
    if printClasses:
      print(f'{cls}:{sum(data[atribute].values==cls)}')
  
  if plotBar:
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    ax.bar([*countClasses.keys()],[*countClasses.values()])
    plt.xlabel('Clase')
    plt.ylabel('Cantidad de registros')
    plt.title('Distribución de Clases')
    plt.show()
  return countClasses

In [6]:
def correlationMatrix(givenDataset,atribute_list):
  # Creating a correlation matrix 
  correlationMatrix  = givenDataset.loc[:,atribute_list].corr()
  # Printing the correlation matrix.
  print(correlationMatrix)
  # Displaying the correlation matrix
  sns.heatmap(correlationMatrix, annot=True)

In [7]:
def extractPreCVPreTest(data,atribute,numValues):
  if numValues % 2 == 0:
    pre_test = data[data['Class'].isin([atribute])].sample(n = int(numValues/2))
    data = data.drop(pre_test.index)
    pre_cv = data[data['Class'].isin([atribute])].sample(n = int(numValues/2))
    data = data.drop(pre_cv.index)
  else:
    pre_test = data[data['Class'].isin([atribute])].sample(n = int(numValues/2))
    data = data.drop(pre_test.index)
    pre_cv = data[data['Class'].isin([atribute])].sample(n = int(numValues/2) + 1)
    data = data.drop(pre_cv.index)
  
  return data, pre_cv, pre_test

In [8]:
def split_train_test(data,numValues):
  test = data.sample(n = numValues)
  data = data.drop(test.index)
  
  return data, test

In [9]:
def cantValuesDataFrame(data, frac):
  a = len(data)
  total = a*frac
  n = int(total/2)
  
  return n

In [10]:
def fiveClasse(data, atributes1, atributes2):
  data_opcion_larga = data[data['Class'].isin(atributes1)]
  data_opcion_ninguno = data[data['Class'].isin(atributes2)]
  data_opcion_ninguno.drop(['Class'], axis =1)
  array = np.full((len(data_opcion_ninguno),1),'OTRO')
  data_opcion_ninguno = data_opcion_ninguno.assign(Class=array)
  data_opcion2 = pd.concat([data_opcion_ninguno,data_opcion_larga])
  data_opcion2 = shuffle(data_opcion2)

  return data_opcion2

In [11]:
def create_and_save(dataset, columns_lista_random):
  i = 1
  for lista in columns_lista_random:
    # Se va a tomar una lista de las randoms y nos quedamos con el dataframe achicado:
    lista.append("Size of the residue")
    dataframe_actual = dataset[lista]
    dataframe_actual.to_csv(f'/content/drive/Shareddrives/Redes de Piluchina/TP 1 de Redes/dataset_rg_lineal/dataset_regresionLineal_{i}.csv', index = False)
    i = i + 1
    

In [12]:
def modeloQDA(df_train,keys):
  enc = LabelEncoder()
  labels = enc.fit_transform(df_train.Class.values)
  clf = QuadraticDiscriminantAnalysis()
  clf.fit(df_train[keys].values, labels)
  scores_f1 = cross_val_score(clf, df_train[keys], labels, cv= 5, scoring = 'f1_micro')
  scores_f2 = cross_val_score(clf, df_train[keys], labels, cv= 5, scoring = 'f1_macro')
  f1_micro = scores_f1.mean()
  f1_macro = scores_f1.mean()

  return clf, f1_micro, f1_macro, enc

In [13]:
def predicctions(df,keys,enc,modelo):
  labels = enc.transform(df.Class.values)
  prediction = modelo.predict(df[keys])

  return prediction, labels

In [14]:
def modeloRegLogic(df_train,keys, C_value, iter_value):
  enc = LabelEncoder()
  labels = enc.fit_transform(df_train.Class.values)
  clf=LogisticRegression(C = C_value, max_iter = iter_value)
  clf.fit(df_train[keys].values, labels)
  scores_f1 = cross_val_score(clf, df_train[keys], labels, cv= 5, scoring = 'f1_micro')
  scores_f2 = cross_val_score(clf, df_train[keys], labels, cv= 5, scoring = 'f1_macro')
  f1_micro = scores_f1.mean()
  f1_macro = scores_f1.mean()

  return clf, f1_micro, f1_macro, enc

In [15]:
def predicctions_RegLogic(df,keys,enc,modelo):
  labels = enc.transform(df.Class.values)
  prediction = modelo.predict(df[keys])

  return prediction, labels

In [16]:
def BIScreatePolFeaturesRegLogic(df,cant_features,keys, C_value, iter_value):
  enc = LabelEncoder()
  labels = enc.fit_transform(df.Class.values)
  # buscamos los features polinómicos de orden 2
  poly = PolynomialFeatures(cant_features)
  new_train = poly.fit_transform(df[keys].values)
  #new_valid = poly.transform(df_valid[keys].values)
  # Aplicamos regresión logística
  clf=LogisticRegression(C=C_value, max_iter = iter_value)
  clf.fit(new_train, labels)

  scores_f1 = cross_val_score(clf, new_train, labels, cv= 5, scoring = 'f1_micro')
  f1_micro = scores_f1.mean()
  scores_f2 = cross_val_score(clf, new_train, labels, cv= 5, scoring = 'f1_macro')
  f1_macro = scores_f2.mean()

  return clf, poly, f1_micro, f1_macro, enc

In [17]:
def predictPolFeatures(df,poly,keys,enc,modelo):
  labels = enc.transform(df.Class.values)
  new_test = poly.transform(df[keys].values)
  prediction = modelo.predict(df[keys])
  return prediction, labels

In [18]:
def plot_roc_curve(fper, tper):
  
    plt.plot(fper, tper, color='red', label='ROC')
    plt.plot([0, 1], [0, 1], color='green', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
      
    plt.legend()
    plt.show()

In [19]:
def RegLinearpolFeatures(degreeValue, df_atributes):
  # Se crean los Polynomial Features
  poly = PolynomialFeatures(degree = degreeValue)
  x_poly = poly.fit_transform(df_atributes.values)

  return x_poly

In [20]:
def entrenarRegLineal(df,degreeValue = 1, ifPoly = True):
  df_atributes = df.drop(columns = 'Size of the residue')
  # Se crean los Polynomial Features
  if ifPoly:
    x_atributes = RegLinearpolFeatures(degreeValue, df_atributes)
    # Se divide el df en train y test:
    X_train, X_test, y_train, y_test = train_test_split(x_atributes,df['Size of the residue'].values,test_size = 0.20, shuffle = True)
  else:
    X_train, X_test, y_train, y_test = train_test_split(df_atributes.values,df['Size of the residue'].values,test_size = 0.20, shuffle = True)
  # Se entrena el modelo:
  clf_lineal = LinearRegression()
  MSE = cross_val_score(clf_lineal, X_train, y_train, cv= 5, scoring = 'neg_mean_squared_error')
  MSE_mean = MSE.mean()
  #clf_lineal.fit(X_train,y_train)

  return X_train, X_test, y_train, y_test, MSE_mean

In [21]:
def saveCositas(df,num_data,arrayDegrees):
  atributos =  list(df.columns)
  # Primero se hace para el df sin polFeatures:
  X_train, X_test, y_train, y_test, MSE_mean = entrenarRegLineal(df, ifPoly = False)
  dic = {'Num_dataset': num_data,'polFeatures': [0],'Atributos': [atributos],'X_train': [X_train], 'X_test': [X_test], 'y_train': [y_train], 'y_test': [y_test], 'MSE_mean': [MSE_mean]}
  df_reaults = pd.DataFrame(dic)

  # Se corre con array de PolFeatures:
  for deg in arrayDegrees:
    X_train, X_test, y_train, y_test, MSE_mean = entrenarRegLineal(df, degreeValue = deg)
    dic = {'Num_dataset': num_data,'polFeatures': [deg],'Atributos': [atributos],'X_train': [X_train], 'X_test': [X_test], 'y_train': [y_train], 'y_test': [y_test], 'MSE_mean': [MSE_mean]}
    df_prev = pd.DataFrame(dic)
    df_reaults= pd.concat([df_reaults,df_prev])
  
  return df_reaults

In [22]:
def evaluarPosibilidades(arrayDegrees):
  df = pd.read_csv(f'/content/drive/Shareddrives/Redes de Piluchina/TP 1 de Redes/dataset_rg_lineal/dataset_regresionLineal_1.csv')
  df_results = saveCositas(df, 1, arrayDegrees = arrayDegrees)
  for value in range(2,51):
    df = pd.read_csv(f'/content/drive/Shareddrives/Redes de Piluchina/TP 1 de Redes/dataset_rg_lineal/dataset_regresionLineal_{value}.csv')
    df_prev = saveCositas(df, value, arrayDegrees = arrayDegrees)
    df_results = pd.concat([df_results,df_prev])
  
  df_results = df_results.reset_index().drop(['index'], axis=1)

  return df_results

In [23]:
def n_mejores(df,n):
  mejores = df['MSE_mean'].nlargest(n)
  Index = list(mejores.index)
  df_mejores = df.loc[Index]
  #mejores_fin = pd.DataFrame({'Index': Index, 'Values': Values})
  df_mejores = df_mejores.reset_index().drop(['index'], axis=1)

  return df_mejores

In [24]:
def RegLinPredicction(mejores):
  df_final = mejores[['Num_dataset',	'polFeatures',	'Atributos', 'MSE_mean']]
  array = []
  for i in range(0,len(mejores)):
    X_train = mejores.iloc[i]['X_train']
    y_train = mejores.iloc[i]['y_train']
    X_test = mejores.iloc[i]['X_test']
    y_test = mejores.iloc[i]['y_test']
    clf_lineal = LinearRegression()
    clf_lineal.fit(X_train,y_train)
    prediccion = clf_lineal.predict(X_test)
    MSE_Test = mean_squared_error(y_test, prediccion)
    array.append(MSE_Test)
  
  dataset = pd.DataFrame({'MSE_TEST': array})
  df_final = pd.concat([df_final,dataset],axis=1)

  return df_final
